# Final model inference

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.3 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip train.csv.zip
!unzip features.csv.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 323MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            


In [3]:
!pip install torch mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q wandb torch torchvision torchaudio -q prophet neuralforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
!unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [6]:
#!/usr/bin/env python3
"""
Walmart Sales Forecasting - Hybrid Model Inference (TFT + Prophet)
Train on full training dataset and generate predictions for test.csv
Adaptive weighting:
- Holiday periods: 60% Prophet + 40% TFT
- Regular days: 65% TFT + 35% Prophet
"""

import warnings
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from prophet import Prophet
from statsmodels.tools.sm_exceptions import ValueWarning
import zipfile
import os
import logging
from datetime import datetime, timedelta
import mlflow
import mlflow.sklearn

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore")
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)
# Additional cmdstanpy suppression
import logging
logging.getLogger('cmdstanpy').disabled = True
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

# Data path configuration
KAGGLE_DATA_PATH = "/kaggle/input/walmart-recruiting-store-sales-forecasting/"

def calculate_wmae(y_true, y_pred, is_holiday_flag, holiday_weight=5.0):
    """Calculate Weighted Mean Absolute Error (WMAE) as per competition rules."""
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday_flag.astype(bool), holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae

# =============================================================================
# TFT Components (from model-exp-tft-fx.ipynb)
# =============================================================================

class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if "Date" not in X.columns:
            raise ValueError("DateFeatureCreator requires 'Date' column in input X.")

        # Ensure 'Date' is datetime type before operations
        if not pd.api.types.is_datetime64_any_dtype(X['Date']):
            X['Date'] = pd.to_datetime(X['Date'])

        # Using to_period('W') and then converting to integer week number
        # rank(method="dense") ensures consecutive integers for weeks
        X["week"] = (X["Date"].dt.to_period("W").rank(method="dense").astype(int) - 1)

        # Cyclical features for different periodicities
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13) # Roughly quarterly seasonality
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23) # A different, less common periodicity
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)

        # Drop the original 'Date' column as its information is now in cyclical features
        X = X.drop(columns=["Date"], errors='ignore')
        return X

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.columns, errors="ignore")

class ColumnTransformerWithNames(ColumnTransformer):
    """
    A wrapper around ColumnTransformer to retain column names and return a DataFrame.
    Handles OneHotEncoder output specifically.
    """
    def __init__(self, transformers, remainder='drop'):
        super().__init__(transformers=transformers, remainder=remainder)
        self.output_columns_ = None

    def fit(self, X, y=None):
        super().fit(X, y)
        self.output_columns_ = self._get_feature_names_out_internal(X)
        return self

    def _get_feature_names_out_internal(self, X):
        column_names = []
        for name, transformer, columns in self.transformers_:
            if transformer == 'drop':
                continue
            elif transformer == 'passthrough':
                # Ensure passthrough columns are correctly identified from original X
                if isinstance(columns, str):
                    column_names.append(columns)
                else:
                    column_names.extend(list(columns))
            else:
                if hasattr(transformer, 'get_feature_names_out'):
                    if isinstance(columns, str):
                        col_names = [columns]
                    else:
                        col_names = list(columns)
                    column_names.extend(list(transformer.get_feature_names_out(col_names)))
                else:
                    if isinstance(columns, str): # Fallback for transformers without get_feature_names_out
                        column_names.append(columns)
                    else:
                        column_names.extend(list(columns))
        return column_names

    def transform(self, X):
        transformed_array = super().transform(X)
        if self.output_columns_ is None:
             raise RuntimeError("ColumnTransformerWithNames must be fitted before transform.")

        # Convert to dense array if it's a sparse matrix (older sklearn default for OHE)
        if hasattr(transformed_array, 'toarray'):
            transformed_array = transformed_array.toarray()

        # Ensure that the index is preserved from the input X
        # This is CRITICAL for maintaining alignment with y
        return pd.DataFrame(transformed_array, index=X.index, columns=self.output_columns_)

    def fit_transform(self, X, y=None):
        transformed_array = super().fit_transform(X, y)
        self.output_columns_ = self._get_feature_names_out_internal(X)

        # Convert to dense array if it's a sparse matrix (older sklearn default for OHE)
        if hasattr(transformed_array, 'toarray'):
            transformed_array = transformed_array.toarray()

        # Ensure that the index is preserved from the input X
        # This is CRITICAL for maintaining alignment with y
        return pd.DataFrame(transformed_array, index=X.index, columns=self.output_columns_)

class MultiIndexKeeper(BaseEstimator, TransformerMixin):
    def __init__(self, index_cols=["Date", "Store", "Dept"]):
        self.index_cols = index_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        if 'Date' in X.columns and not pd.api.types.is_datetime64_any_dtype(X['Date']):
            X['Date'] = pd.to_datetime(X['Date'])

        missing_cols = [col for col in self.index_cols if col not in X.columns]
        if missing_cols:
            raise ValueError(f"MultiIndexKeeper: Missing columns in input X: {missing_cols}")

        # IMPORTANT: When setting index, ensure 'Date', 'Store', 'Dept' columns
        # are not dropped, as they are needed later by NeuralForecast as covariates.
        # This is already handled by drop=False.
        X.set_index(self.index_cols, drop=False, inplace=True)
        return X

class TFTRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_chunk_length=52, output_chunk_length=39, epochs=25, batch_size=32, random_seed=42):
        self.input_chunk_length = input_chunk_length
        self.output_chunk_length = output_chunk_length
        self.epochs = epochs
        self.batch_size = batch_size
        self.random_seed = random_seed
        self.nf_ = None
        self.model_ = None
        self.trained_df_ = None # Store the DataFrame used for training

    def fit(self, X, y):
        # Ensure y has a name for proper merging if it doesn't already
        if y.name is None:
            y.name = 'y'

        y_multiindexed = pd.Series(y.values, index=X.index, name='y')

        df = X.copy()
        df['y'] = y_multiindexed

        if not pd.api.types.is_datetime64_any_dtype(df.index.get_level_values('Date')):
            raise ValueError("MultiIndex 'Date' level is not datetime type. Ensure MultiIndexKeeper makes it datetime.")
        df['ds'] = df.index.get_level_values('Date')

        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        # Store the prepared DataFrame for use in predict
        self.trained_df_ = df.copy() # Store the full df (including y and features)

        # --- DEBUGGING STEP: Check for NaNs immediately before NeuralForecast fit ---
        nan_check = df.isnull().sum()
        cols_with_nans = nan_check[nan_check > 0].index.tolist()
        if cols_with_nans:
            print(f"DEBUG: Found NaNs in the following columns before NeuralForecast fit: {cols_with_nans}")
            # Print head including index for better context
            print(df.loc[df[cols_with_nans[0]].isnull(), cols_with_nans].head())
            raise ValueError(f"Found missing values in {cols_with_nans}.")
        # --- END DEBUGGING STEP ---

        self.model_ = TFT(
            h=self.output_chunk_length,
            input_size=self.input_chunk_length,
            batch_size=self.batch_size,
            random_seed=self.random_seed,
        )

        self.nf_ = NeuralForecast(models=[self.model_], freq="W-FRI")

        self.nf_.fit(df=df)
        return self

    def predict(self, X):
        # X here is the transformed X_test from the pipeline, with MultiIndex ('Date', 'Store', 'Dept')

        # 1. Prepare future covariates from X (which is X_test after preprocessing)
        df_future_covariates_raw = X.copy() # This X contains all features for the test period

        if not pd.api.types.is_datetime64_any_dtype(df_future_covariates_raw.index.get_level_values('Date')):
            raise ValueError("MultiIndex 'Date' level is not datetime type in predict. Ensure MultiIndexKeeper makes it datetime.")

        df_future_covariates_raw['ds'] = df_future_covariates_raw.index.get_level_values('Date')
        df_future_covariates_raw["unique_id"] = df_future_covariates_raw["Store"].astype(str) + "_" + df_future_covariates_raw["Dept"].astype(str)

        # Identify all covariate columns (all columns in self.trained_df_ except 'ds', 'unique_id', 'y')
        covariate_cols = [col for col in self.trained_df_.columns if col not in ['ds', 'unique_id', 'y']]

        # Select only the relevant future covariate columns and the required 'ds', 'unique_id'
        df_future_covariates_selected = df_future_covariates_raw[['ds', 'unique_id'] + covariate_cols].copy()

        # 2. Generate the full expected future dataframe for all series for the forecast horizon
        expected_future_df_template = self.nf_.make_future_dataframe(self.trained_df_)

        # 3. Merge the generated template with our actual future covariates (X_test)
        futr_df_complete = pd.merge(
            expected_future_df_template,
            df_future_covariates_selected,
            on=['unique_id', 'ds'],
            how='left'
        )

        nan_check_futr = futr_df_complete.isnull().sum()
        cols_with_nans_futr = nan_check_futr[nan_check_futr > 0].index.tolist()
        if cols_with_nans_futr:
            print(f"DEBUG: Found NaNs in futr_df_complete for columns: {cols_with_nans_futr}. Filling with 0.")
            futr_df_complete[cols_with_nans_futr] = futr_df_complete[cols_with_nans_futr].fillna(0)

        # 4. Perform the prediction
        # Ensure that `df` has all required unique_ids and `futr_df` aligns.
        # This is the point where the model generates predictions.
        forecast_df = self.nf_.predict(df=self.trained_df_, futr_df=futr_df_complete)

        forecast_df = forecast_df.rename(columns={'TFT': 'yhat'})

        if 'unique_id' in forecast_df.columns:
            forecast_df[['Store', 'Dept']] = forecast_df['unique_id'].str.split('_', expand=True)
            forecast_df['Store'] = forecast_df['Store'].astype(float).astype(int)
            forecast_df['Dept'] = forecast_df['Dept'].astype(float).astype(int)

        forecast_df['Date'] = pd.to_datetime(forecast_df['ds'])

        # Ensure the index columns are correct before setting the index
        # Also, make sure that 'Store' and 'Dept' are properly integer type before setting multi-index
        forecast_df_indexed = forecast_df.set_index(['Date', 'Store', 'Dept'])[['yhat']]

        # This is where NaNs can be introduced if forecast_df_indexed doesn't cover all X.index
        final_predictions = forecast_df_indexed.reindex(X.index)

        y_pred = final_predictions['yhat'].values.flatten()

        # FIX: Fill any NaNs in the final predictions array with 0 before evaluation
        if np.isnan(y_pred).any():
            print("DEBUG: Found NaNs in final y_pred after reindex. Filling with 0.")
            y_pred = np.nan_to_num(y_pred, nan=0.0)

        y_pred[y_pred < 0] = 0

        return y_pred

# =============================================================================
# Prophet Components (from model_exp_FX_Prophet.ipynb)
# =============================================================================

class WalmartProphetInferencePreprocessingPipeline:
    """
    Preprocessing pipeline for Prophet models - Inference version.
    Focuses on preparing data in the 'ds' (Date) and 'y' (Weekly_Sales) format,
    and handling holidays for inference.
    """

    def __init__(self):
        self.fitted = False
        self.holidays_df = None

    def load_and_prepare_data(self):
        """Load and merge necessary datasets for Prophet inference."""
        print("📊 Loading datasets for inference...")

        # Load datasets
        train_df = pd.read_csv('train.csv')
        test_df = pd.read_csv('test.csv')
        features_df = pd.read_csv('features.csv')
        stores_df = pd.read_csv('stores.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   📉 Test data: {test_df.shape}")
        print(f"   📊 Features data: {features_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")

        # Convert Date columns to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        test_df['Date'] = pd.to_datetime(test_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        # Merge train datasets (similar to previous steps)
        merged_train = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')
        train_full = pd.merge(merged_train, stores_df, on=['Store'], how='left')

        # Merge test datasets (test.csv doesn't have Weekly_Sales or IsHoliday)
        merged_test = pd.merge(test_df, features_df, on=['Store', 'Date'], how='left')
        test_full = pd.merge(merged_test, stores_df, on=['Store'], how='left')

        # Sort by date for time series consistency
        train_full = train_full.sort_values(by=['Store', 'Dept', 'Date']).reset_index(drop=True)
        test_full = test_full.sort_values(by=['Store', 'Dept', 'Date']).reset_index(drop=True)

        print(f"   ✅ Merged train data: {train_full.shape}")
        print(f"   ✅ Merged test data: {test_full.shape}")
        print(f"   📅 Train date range: {train_full['Date'].min()} to {train_full['Date'].max()}")
        print(f"   📅 Test date range: {test_full['Date'].min()} to {test_full['Date'].max()}")

        return train_full, test_full

    def fit(self, train_data):
        """Fit the preprocessing pipeline (prepare holidays)."""
        print("🔧 Preparing Prophet specific data (holidays) for inference...")

        # Prophet's holidays DataFrame: requires 'holiday', 'ds' columns
        # We define common US holidays that align with Walmart's IsHoliday flag
        # Extended to cover test period as well
        self.holidays_df = pd.DataFrame([
            # Super Bowl: IsHoliday=True
            {'holiday': 'SuperBowl', 'ds': '2010-02-12'},
            {'holiday': 'SuperBowl', 'ds': '2011-02-11'},
            {'holiday': 'SuperBowl', 'ds': '2012-02-10'},
            {'holiday': 'SuperBowl', 'ds': '2013-02-08'},

            # Labor Day: IsHoliday=True
            {'holiday': 'LaborDay', 'ds': '2010-09-10'},
            {'holiday': 'LaborDay', 'ds': '2011-09-09'},
            {'holiday': 'LaborDay', 'ds': '2012-09-07'},
            {'holiday': 'LaborDay', 'ds': '2013-09-06'},

            # Thanksgiving: IsHoliday=True
            {'holiday': 'Thanksgiving', 'ds': '2010-11-26'},
            {'holiday': 'Thanksgiving', 'ds': '2011-11-25'},
            {'holiday': 'Thanksgiving', 'ds': '2012-11-23'},
            {'holiday': 'Thanksgiving', 'ds': '2013-11-29'},

            # Christmas: IsHoliday=True
            {'holiday': 'Christmas', 'ds': '2010-12-31'},
            {'holiday': 'Christmas', 'ds': '2011-12-30'},
            {'holiday': 'Christmas', 'ds': '2012-12-28'},
            {'holiday': 'Christmas', 'ds': '2013-12-27'}
        ])

        # Convert ds to datetime
        self.holidays_df['ds'] = pd.to_datetime(self.holidays_df['ds'])

        self.fitted = True
        print("✅ Pipeline fitted on training data with holiday-aware settings for inference")

        return self

    def transform(self, data, is_test=False):
        """Transform data for Prophet format."""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transforming data.")

        print("🔄 Transforming training data..." if not is_test else "🔄 Transforming test data...")

        # Prophet requires very specific column names: 'ds', 'y'
        prophet_data = data.copy()

        # For training data
        if not is_test:
            # Ensure we have all necessary columns
            required_cols = ['Date', 'Weekly_Sales', 'Store', 'Dept', 'IsHoliday']
            if not all(col in prophet_data.columns for col in required_cols):
                missing = [col for col in required_cols if col not in required_cols]
                raise ValueError(f"Missing required columns: {missing}")

            # Rename columns for Prophet
            prophet_data = prophet_data.rename(columns={
                'Date': 'ds',
                'Weekly_Sales': 'y'
            })

            # Keep only necessary columns for Prophet
            prophet_data = prophet_data[['ds', 'y', 'Store', 'Dept', 'IsHoliday']].copy()
        else:
            # For test data
            required_cols = ['Date', 'Store', 'Dept']
            if not all(col in prophet_data.columns for col in required_cols):
                missing = [col for col in required_cols if col not in prophet_data.columns]
                raise ValueError(f"Missing required columns: {missing}")

            # Rename columns for Prophet
            prophet_data = prophet_data.rename(columns={
                'Date': 'ds'
            })

            # Keep only necessary columns for Prophet
            available_cols = ['ds', 'Store', 'Dept']
            if 'IsHoliday' in prophet_data.columns:
                available_cols.append('IsHoliday')
            prophet_data = prophet_data[available_cols].copy()

        # Ensure ds is datetime
        prophet_data['ds'] = pd.to_datetime(prophet_data['ds'])

        print(f"✅ Transform complete. Shape: {prophet_data.shape}")

        return prophet_data

    def fit_transform(self, train_data):
        """Convenience method to fit and transform in one step."""
        return self.fit(train_data).transform(train_data, is_test=False)

    def get_preprocessed_data(self):
        """
        Orchestrates preprocessing steps to get model-ready data for inference.

        Returns:
            train_data_prophet: DataFrame ready for Prophet training
            test_data_prophet: DataFrame ready for Prophet testing
            holidays_df: DataFrame of holidays for Prophet
        """
        print("🔄 Getting preprocessed data for inference...")

        # Create the preprocessing pipeline
        pipeline = WalmartProphetInferencePreprocessingPipeline()

        # Load raw data
        train_full, test_full = pipeline.load_and_prepare_data()

        # Fit and transform data using pipeline
        pipeline.fit(train_full)
        train_data_prophet = pipeline.transform(train_full, is_test=False)
        test_data_prophet = pipeline.transform(test_full, is_test=True)

        holidays_df = pipeline.holidays_df # Get holidays after fitting pipeline

        print(f"✅ Data preprocessing complete!")
        print(f"   📊 Training shape: {train_data_prophet.shape}")
        print(f"   📊 Test shape: {test_data_prophet.shape}")

        return train_data_prophet, test_data_prophet, holidays_df

def train_prophet_models_inference(train_data_prophet, holidays_df, min_observations=50):
    """
    Train Prophet models for each Store-Dept combination for inference.

    Args:
        train_data_prophet: Training data in Prophet format (ds, y, Store, Dept, IsHoliday)
        holidays_df: DataFrame of holidays for Prophet
        min_observations: Minimum number of observations required to train a model

    Returns:
        dict: Dictionary with (Store, Dept) as keys and trained Prophet models as values
    """
    print("📈 Training Prophet models for inference...")

    # Get unique Store-Dept combinations
    unique_combinations = train_data_prophet[['Store', 'Dept']].drop_duplicates()
    total_combinations = len(unique_combinations)

    print(f"   📊 Training models for {total_combinations} combinations on full dataset")
    print(f"   🎯 Training Prophet for all combinations")

    models = {}
    successful_count = 0
    failed_count = 0

    for idx, (_, row) in enumerate(unique_combinations.iterrows(), 1):
        store_id = row['Store']
        dept_id = row['Dept']

        # Filter data for this specific Store-Dept combination
        series_data = train_data_prophet[
            (train_data_prophet['Store'] == store_id) &
            (train_data_prophet['Dept'] == dept_id)
        ].copy()

        # Check if we have enough data points
        if len(series_data) < min_observations:
            failed_count += 1
            continue

        try:
            # Initialize Prophet model with holidays
            m = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                holidays=holidays_df
            )

            # Train the model on this series
            # Prophet expects only 'ds' and 'y' columns for fitting
            fit_data = series_data[['ds', 'y']].copy()
            m.fit(fit_data)

            # Store the trained model
            models[(store_id, dept_id)] = m
            successful_count += 1

        except Exception as e:
            # If training fails for this series, skip it
            failed_count += 1
            continue

        # Progress update every 200 models
        if idx % 200 == 0 or idx == 1:
            print(f"   ✅ Trained {idx}/{total_combinations} models ({successful_count} successful, {failed_count} failed)")

    print(f"   ✅ Trained {total_combinations}/{total_combinations} models ({successful_count} successful, {failed_count} failed)")

    print("✅ Prophet training complete!")
    print(f"   🎯 Successful models: {successful_count}")
    print(f"   ❌ Failed models: {failed_count}")
    print(f"   📊 Coverage: {successful_count}/{total_combinations} ({successful_count/total_combinations*100:.1f}%)")

    return models

def make_prophet_predictions_inference(models, test_data_prophet):
    """
    Make predictions using trained Prophet models for inference.

    Args:
        models: Dictionary of trained Prophet models
        test_data_prophet: Test data in Prophet format

    Returns:
        tuple: (predictions, store_dept_date_info) as numpy arrays and dataframe
    """
    print("📈 Making Prophet predictions for inference...")

    predictions = []
    store_info = []
    dept_info = []
    date_info = []

    successful_predictions_count = 0
    skipped_predictions_no_model = 0
    failed_predictions_count = 0

    # Debug: Count total test points
    total_test_points = len(test_data_prophet)
    print(f"   📊 Total test points: {total_test_points}")
    print(f"   📊 Available models: {len(models)}")

    # Get unique Store-Dept combinations from test data
    unique_combinations = test_data_prophet[['Store', 'Dept']].drop_duplicates()
    print(f"   📊 Unique Store-Dept combinations in test: {len(unique_combinations)}")

    combinations_with_models = 0
    combinations_without_models = 0

    for _, row in unique_combinations.iterrows():
        store_id = row['Store']
        dept_id = row['Dept']

        # Filter test data for this Store-Dept combination
        series_data = test_data_prophet[
            (test_data_prophet['Store'] == store_id) &
            (test_data_prophet['Dept'] == dept_id)
        ].copy()

        if series_data.empty:
            continue

        # Check if we have a trained model for this combination
        if (store_id, dept_id) not in models:
            # No model available - use zeros as predictions
            predictions.extend(np.zeros(len(series_data)))
            store_info.extend([store_id] * len(series_data))
            dept_info.extend([dept_id] * len(series_data))
            date_info.extend(series_data['ds'].tolist())
            skipped_predictions_no_model += len(series_data)
            combinations_without_models += 1
            continue

        combinations_with_models += 1

        try:
            # Get the trained model
            m = models[(store_id, dept_id)]

            # Make predictions for this series
            # Prophet needs only 'ds' column for prediction
            forecast = m.predict(series_data[['ds']])

            # Extract predictions and ensure no negative values
            yhat = forecast['yhat'].values
            yhat[yhat < 0] = 0  # Ensure no negative predictions

            # Store results
            predictions.extend(yhat)
            store_info.extend([store_id] * len(series_data))
            dept_info.extend([dept_id] * len(series_data))
            date_info.extend(series_data['ds'].tolist())
            successful_predictions_count += len(series_data)

        except Exception as e:
            # If prediction fails, use zeros
            predictions.extend(np.zeros(len(series_data)))
            store_info.extend([store_id] * len(series_data))
            dept_info.extend([dept_id] * len(series_data))
            date_info.extend(series_data['ds'].tolist())
            failed_predictions_count += len(series_data)

    print("✅ Predictions complete!")
    print(f"   🎯 Prophet predictions: {successful_predictions_count}")
    print(f"   ⏭️ Skipped (no model): {skipped_predictions_no_model}")
    print(f"   ❌ Failed predictions: {failed_predictions_count}")
    print(f"   📊 Combinations with models: {combinations_with_models}")
    print(f"   📊 Combinations without models: {combinations_without_models}")
    print(f"   📊 Total predictions generated: {len(predictions)}")

    # Create info DataFrame
    info_df = pd.DataFrame({
        'Store': store_info,
        'Dept': dept_info,
        'Date': date_info
    })

    return np.array(predictions), info_df

def run_standalone_prophet_inference():
    """
    Run the Prophet experiment for inference to get Prophet predictions on test data.

    Returns:
        tuple: (prophet_predictions, prediction_info_df)
    """
    print("🚀 Running Standalone Prophet Inference")
    print("=" * 80)

    try:
        # Step 1: Get preprocessed data for inference
        print("\n📊 Step 1: Data preprocessing for inference...")
        train_data_prophet, test_data_prophet, holidays_df = WalmartProphetInferencePreprocessingPipeline().get_preprocessed_data()

        # Step 2: Train Prophet models on full training data
        print("\n📈 Step 2: Training Prophet models on full training data...")
        models = train_prophet_models_inference(train_data_prophet, holidays_df)

        # Step 3: Make predictions on test set
        print("\n📈 Step 3: Making predictions on test data...")
        prophet_predictions, prediction_info = make_prophet_predictions_inference(models, test_data_prophet)

        print("\n✅ Standalone Prophet Inference: COMPLETE!")
        print(f"   📊 Generated {len(prophet_predictions):,} predictions")

        return prophet_predictions, prediction_info

    except Exception as e:
        print(f"❌ Standalone Prophet inference failed: {e}")
        raise

# =============================================================================
# Hybrid Model Implementation
# =============================================================================

class HybridTFTProphetInferenceRegressor(BaseEstimator, RegressorMixin):
    """
    Hybrid model combining TFT and Prophet with adaptive weighting for inference:
    - Holiday periods: 60% Prophet + 40% TFT
    - Regular days: 65% TFT + 35% Prophet
    """

    def __init__(self, input_chunk_length=52, output_chunk_length=39, epochs=25, batch_size=32,
                 random_seed=42, min_observations=50):
        # TFT parameters
        self.input_chunk_length = input_chunk_length
        self.output_chunk_length = output_chunk_length
        self.epochs = epochs
        self.batch_size = batch_size
        self.random_seed = random_seed

        # Prophet parameters
        self.min_observations = min_observations

        # Model components
        self.tft_model_ = None
        self.prophet_predictions_ = None  # Store Prophet predictions from inference
        self.prediction_info_ = None      # Store prediction info (Store, Dept, Date)

        # Test data for TFT prediction
        self.X_test_full_ = None

        # Hybrid weights
        self.holiday_prophet_weight = 0.60
        self.holiday_tft_weight = 0.40
        self.regular_tft_weight = 0.65
        self.regular_prophet_weight = 0.35

    def fit(self, X, y):
        """Fit Prophet first on full training data, then train TFT on the same data."""
        print("🤖 Training Hybrid Model for Inference (TFT + Prophet)...")

        # STEP 1: Run standalone Prophet inference to get model trained
        print("\n📊 Step 1: Running standalone Prophet inference...")

        prophet_predictions, prediction_info = run_standalone_prophet_inference()

        # Store Prophet results for later use in predict
        self.prophet_predictions_ = prophet_predictions
        self.prediction_info_ = prediction_info

        print(f"\n✅ Prophet inference preparation complete!")
        print(f"   📊 Prophet test predictions: {len(prophet_predictions):,}")

        # STEP 2: Train TFT on full training data
        print("\n📊 Step 2: Training TFT on full training data...")

        self.tft_model_ = TFTRegressor(
            input_chunk_length=self.input_chunk_length,
            output_chunk_length=self.output_chunk_length,
            epochs=self.epochs,
            batch_size=self.batch_size,
            random_seed=self.random_seed
        )

        # Train TFT on full training data
        self.tft_model_.fit(X, y)

        print("✅ TFT model training complete!")
        print("✅ Hybrid model training complete!")

        return self

    def predict(self, X):
        """Make hybrid predictions using both models."""
        print("🔮 Making hybrid predictions for inference...")

        # Store test data for TFT prediction
        self.X_test_full_ = X.copy()

        # Check if we have Prophet predictions from training
        if self.prophet_predictions_ is None:
            raise ValueError("Prophet predictions not available. Train the model first.")

        # STEP 1: Use pre-computed Prophet predictions from inference
        print("   📊 Using Prophet inference predictions...")
        prophet_predictions = self.prophet_predictions_
        prediction_info = self.prediction_info_

        print(f"   📊 Prophet predictions: {len(prophet_predictions):,}")

        # STEP 2: Get TFT predictions on test data
        print("   📊 Getting TFT predictions on test data...")

        tft_predictions = self.tft_model_.predict(X)
        print(f"   📊 TFT predictions: {len(tft_predictions):,}")

        # STEP 3: Verify alignment (should be the same length)
        print(f"   ✅ Checking alignment - Prophet: {len(prophet_predictions)}, TFT: {len(tft_predictions)}")

        if len(prophet_predictions) != len(tft_predictions):
            print(f"   ⚠️ Length mismatch - Prophet: {len(prophet_predictions)}, TFT: {len(tft_predictions)}")
            # Truncate to minimum length
            min_len = min(len(prophet_predictions), len(tft_predictions))
            prophet_predictions = prophet_predictions[:min_len]
            tft_predictions = tft_predictions[:min_len]
            prediction_info = prediction_info.iloc[:min_len].copy()
            print(f"   ✂️ Truncated to length: {min_len}")
        else:
            print(f"   🎯 Perfect alignment achieved: {len(prophet_predictions)} predictions")

        # STEP 4: Determine holiday flags from features or dates
        print("   📅 Determining holiday flags...")

        # Try to get holiday information from features
        is_holiday = np.zeros(len(prophet_predictions), dtype=bool)

        # Check if IsHoliday is available in the original test data
        if hasattr(self, 'X_test_full_') and 'IsHoliday' in self.X_test_full_.columns:
            # Extract IsHoliday from the original test features
            is_holiday_series = self.X_test_full_['IsHoliday'].reset_index(drop=True)
            if len(is_holiday_series) == len(prophet_predictions):
                is_holiday = is_holiday_series.astype(bool).values
                print(f"   📅 Using IsHoliday from test features: {is_holiday.sum()} holiday periods found")
            else:
                print(f"   ⚠️ IsHoliday length mismatch ({len(is_holiday_series)} vs {len(prophet_predictions)}), using date-based holiday detection")
        else:
            print(f"   📅 IsHoliday not available in test features, using date-based holiday detection")

        # If we couldn't get holiday flags from features, use date-based detection
        if not is_holiday.any():
            # Define holiday dates for the test period (2012-2013)
            holiday_dates = [
                '2012-11-23',  # Thanksgiving 2012
                '2012-11-30',  # Black Friday 2012 (week after Thanksgiving)
                '2012-12-28',  # Christmas 2012
                '2013-02-08',  # Super Bowl 2013
                '2013-09-06',  # Labor Day 2013
                '2013-11-29',  # Thanksgiving 2013
                '2013-12-06',  # Week before Christmas 2013
                '2013-12-27'   # Christmas 2013
            ]
            holiday_dates = pd.to_datetime(holiday_dates)

            # Check if prediction dates match holiday dates
            if len(prediction_info) == len(prophet_predictions):
                pred_dates = pd.to_datetime(prediction_info['Date'])
                is_holiday = pred_dates.isin(holiday_dates).values
                print(f"   📅 Found {is_holiday.sum()} holiday predictions using date matching from {len(holiday_dates)} defined holidays")
            else:
                print(f"   ⚠️ Cannot match dates for holiday detection, using all regular weights")
                is_holiday = np.zeros(len(prophet_predictions), dtype=bool)

        # STEP 5: Apply adaptive weighting
        print("   🎯 Applying adaptive weighting...")

        # Initialize hybrid predictions
        hybrid_predictions = np.zeros_like(tft_predictions, dtype=float)

        # Apply different weights for holidays vs regular days
        holiday_mask = is_holiday
        regular_mask = ~is_holiday

        # Holiday periods: 60% Prophet + 40% TFT
        if np.any(holiday_mask):
            hybrid_predictions[holiday_mask] = (
                self.holiday_prophet_weight * prophet_predictions[holiday_mask] +
                self.holiday_tft_weight * tft_predictions[holiday_mask]
            )

        # Regular days: 65% TFT + 35% Prophet
        if np.any(regular_mask):
            hybrid_predictions[regular_mask] = (
                self.regular_tft_weight * tft_predictions[regular_mask] +
                self.regular_prophet_weight * prophet_predictions[regular_mask]
            )

        # Ensure no negative predictions
        hybrid_predictions[hybrid_predictions < 0] = 0

        # Report weighting statistics
        n_holiday = np.sum(holiday_mask)
        n_regular = np.sum(regular_mask)
        total_points = len(hybrid_predictions)

        print(f"   📊 Weighting applied:")
        print(f"      Holiday points: {n_holiday:,} ({n_holiday/total_points*100:.1f}%) - 60% Prophet + 40% TFT")
        print(f"      Regular points: {n_regular:,} ({n_regular/total_points*100:.1f}%) - 65% TFT + 35% Prophet")

        print("✅ Hybrid predictions complete!")

        return hybrid_predictions

# =============================================================================
# STEP-BY-STEP FUNCTIONS FOR COLAB EXECUTION
# =============================================================================

def step1_load_and_prepare_data():
    """
    Step 1: Load and prepare all data
    Returns: train_full, test_full
    """
    print("📊 STEP 1: Loading and preparing data...")
    print("=" * 50)

    # Check if running in Kaggle environment
    if os.path.exists(KAGGLE_DATA_PATH):
        # Kaggle environment
        train_zip_path = os.path.join(KAGGLE_DATA_PATH, 'train.csv.zip')
        test_zip_path = os.path.join(KAGGLE_DATA_PATH, 'test.csv.zip')
        features_zip_path = os.path.join(KAGGLE_DATA_PATH, 'features.csv.zip')
        stores_csv_path = os.path.join(KAGGLE_DATA_PATH, 'stores.csv')

        print("   📂 Unzipping necessary data files...")
        with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"      - Extracted: {train_zip_path}")

        with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"      - Extracted: {test_zip_path}")

        with zipfile.ZipFile(features_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"      - Extracted: {features_zip_path}")

        stores_df = pd.read_csv(stores_csv_path)
    else:
        # Local environment - assume files are in current directory
        print("   📂 Loading from local files...")
        if os.path.exists('train.csv.zip'):
            with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        if os.path.exists('test.csv.zip'):
            with zipfile.ZipFile('test.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        if os.path.exists('features.csv.zip'):
            with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        stores_df = pd.read_csv('stores.csv')

    # Load the unzipped CSVs
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    features_df = pd.read_csv('features.csv')

    # Convert Date columns to datetime early for consistency
    train_df['Date'] = pd.to_datetime(train_df['Date'])
    test_df['Date'] = pd.to_datetime(test_df['Date'])
    features_df['Date'] = pd.to_datetime(features_df['Date'])

    print(f"   📈 Train data: {train_df.shape}")
    print(f"   📉 Test data: {test_df.shape}")
    print(f"   📊 Features data: {features_df.shape}")
    print(f"   🏪 Stores data: {stores_df.shape}")

    # --- Data Merging and Initial Cleaning ---
    print("\n🧹 Merging training data and initial cleaning...")
    merged_train = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')
    train_full = pd.merge(merged_train, stores_df, on=['Store'], how='left')

    # Merge test data (test.csv doesn't have Weekly_Sales or IsHoliday)
    print("🧹 Merging test data...")
    merged_test = pd.merge(test_df, features_df, on=['Store', 'Date'], how='left')
    test_full = pd.merge(merged_test, stores_df, on=['Store'], how='left')

    # Fill NaN in MarkDown columns with 0, assuming no markdown if not specified
    markdown_cols = [f'MarkDown{i}' for i in range(1, 6)]
    for col in markdown_cols:
        if col in train_full.columns:
            train_full[col] = train_full[col].fillna(0)
        if col in test_full.columns:
            test_full[col] = test_full[col].fillna(0)

    # Remove rows with negative Weekly_Sales from training data
    initial_rows = len(train_full)
    train_full = train_full[train_full['Weekly_Sales'] > 0]
    print(f"   🗑️ Removed {initial_rows - len(train_full)} rows with negative Weekly_Sales from training.")

    # Add IsHoliday to test data if missing
    if "IsHoliday" in train_full.columns and "IsHoliday" not in test_full.columns:
        print("   ⚠️ Adding missing IsHoliday column to test data with default value False")
        test_full['IsHoliday'] = False

    # Ensure continuous series and fill NaNs for NeuralForecast (training data)
    print("   Filling missing dates and sales for time series continuity in training data...")

    # Create a full set of (Store, Dept, Date) unique combinations for training
    unique_store_dept_dates = train_full[['Store', 'Dept', 'Date']].drop_duplicates()

    # Generate all expected dates for each (Store, Dept) in training
    df_list = []
    for (store, dept), group in unique_store_dept_dates.groupby(['Store', 'Dept']):
        series_min_date = group['Date'].min()
        series_max_date = group['Date'].max()
        full_series_dates = pd.date_range(start=series_min_date, end=series_max_date, freq='W-FRI')

        temp_df = pd.DataFrame({
            'Store': store,
            'Dept': dept,
            'Date': full_series_dates
        })
        df_list.append(temp_df)

    complete_series_df = pd.concat(df_list, ignore_index=True)

    # Merge the complete series dates with the original train_full data
    train_full_cleaned = pd.merge(
        complete_series_df,
        train_full,
        on=['Store', 'Dept', 'Date'],
        how='left'
    )

    # Fill NaNs in 'Weekly_Sales' with 0
    nan_sales_before_fill = train_full_cleaned['Weekly_Sales'].isnull().sum()
    train_full_cleaned['Weekly_Sales'] = train_full_cleaned['Weekly_Sales'].fillna(0)
    print(f"   Filled {nan_sales_before_fill} NaN Weekly_Sales values with 0 for series continuity.")

    # Re-merge the original features_df and stores_df to fill in associated data
    train_full_cleaned = pd.merge(train_full_cleaned, features_df.drop(columns=['IsHoliday'], errors='ignore'), on=['Store', 'Date'], how='left', suffixes=('', '_feats'))
    train_full_cleaned = pd.merge(train_full_cleaned, stores_df, on=['Store'], how='left', suffixes=('', '_stores'))

    # Combine IsHoliday if it was duplicated
    if 'IsHoliday_feats' in train_full_cleaned.columns:
        train_full_cleaned['IsHoliday'] = train_full_cleaned['IsHoliday'].fillna(train_full_cleaned['IsHoliday_feats'])
        train_full_cleaned = train_full_cleaned.drop(columns=['IsHoliday_feats'])

    # Define numerical columns for filling NaNs
    numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]

    # Fill NaNs in features columns for training data
    for col in numerical_cols:
        if col in train_full_cleaned.columns:
            train_full_cleaned[col] = train_full_cleaned.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
            train_full_cleaned[col] = train_full_cleaned[col].fillna(train_full_cleaned[col].mean())

    # Fill NaNs in features columns for test data
    for col in numerical_cols:
        if col in test_full.columns:
            test_full[col] = test_full[col].fillna(test_full[col].mean())

    # Ensure no negative sales after any filling process
    train_full_cleaned['Weekly_Sales'][train_full_cleaned['Weekly_Sales'] < 0] = 0

    # Sort by date, store, and department for time series consistency
    train_full = train_full_cleaned.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)
    test_full = test_full.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)

    print(f"   ✅ Merged and cleaned training data: {train_full.shape}")
    print(f"   ✅ Merged and cleaned test data: {test_full.shape}")
    print(f"   📅 Train date range: {train_full['Date'].min()} to {train_full['Date'].max()}")
    print(f"   📅 Test date range: {test_full['Date'].min()} to {test_full['Date'].max()}")

    print("✅ STEP 1 COMPLETE!")
    return train_full, test_full

def step2_train_prophet_and_predict(train_full, test_full):
    """
    Step 2: Train Prophet models and generate predictions
    Returns: prophet_predictions, prediction_info
    """
    print("\n📈 STEP 2: Training Prophet and generating predictions...")
    print("=" * 50)

    # Run standalone Prophet inference
    prophet_predictions, prediction_info = run_standalone_prophet_inference()

    print("✅ STEP 2 COMPLETE!")
    print(f"   📊 Generated {len(prophet_predictions):,} Prophet predictions")

    return prophet_predictions, prediction_info

def step3_prepare_tft_data(train_full, test_full):
    """
    Step 3: Prepare data for TFT training
    Returns: X_train_full, y_train_full, X_test
    """
    print("\n⚙️ STEP 3: Preparing data for TFT...")
    print("=" * 50)

    # Prepare full training set (no split)
    X_train_full = train_full.drop(columns=['Weekly_Sales']).copy()
    y_train_full = train_full['Weekly_Sales'].copy()

    # Test set (no target variable)
    X_test = test_full.copy()

    print(f"   📈 Full training set: {len(X_train_full):,} records")
    print(f"   📉 Test set: {len(X_test):,} records")

    print("✅ STEP 3 COMPLETE!")
    return X_train_full, y_train_full, X_test

def step4_build_tft_pipeline():
    """
    Step 4: Build TFT preprocessing pipeline
    Returns: pipeline
    """
    print("\n🔧 STEP 4: Building TFT preprocessing pipeline...")
    print("=" * 50)

    # Define column lists
    numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]
    categorical_ohe_cols = ["Type", "IsHoliday"]  # IsHoliday should be available now
    passthrough_cols = ["Store", "Dept"]

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean'))
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformerWithNames(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_ohe_cols),
        ('pass', 'passthrough', passthrough_cols)
    ], remainder='drop')

    pipeline = Pipeline([
        ("multi_index_keeper", MultiIndexKeeper(index_cols=["Date", "Store", "Dept"])),
        ("date_feature_creator", DateFeatureCreator()),
        ("preprocessor", preprocessor),
        ("tft_regressor", TFTRegressor(
            input_chunk_length=52,
            output_chunk_length=39,
            epochs=25,
            batch_size=32,
            random_seed=42
        ))
    ])

    print("   ✅ Pipeline built successfully")
    print("✅ STEP 4 COMPLETE!")
    return pipeline

def step5_train_tft(pipeline, X_train_full, y_train_full):
    """
    Step 5: Train TFT model
    Returns: trained pipeline
    """
    print("\n🧠 STEP 5: Training TFT model...")
    print("=" * 50)

    pipeline.fit(X_train_full, y_train_full)

    print("✅ STEP 5 COMPLETE!")
    print("   🎯 TFT model training finished!")
    return pipeline

def step6_predict_tft(pipeline, X_test):
    """
    Step 6: Generate TFT predictions
    Returns: tft_predictions
    """
    print("\n🔮 STEP 6: Generating TFT predictions...")
    print("=" * 50)

    tft_predictions = pipeline.predict(X_test)

    print("✅ STEP 6 COMPLETE!")
    print(f"   📊 Generated {len(tft_predictions):,} TFT predictions")
    return tft_predictions

def step7_create_hybrid_predictions(prophet_predictions, tft_predictions, prediction_info):
    """
    Step 7: Create hybrid predictions with adaptive weighting
    Returns: hybrid_predictions
    """
    print("\n🤝 STEP 7: Creating hybrid predictions...")
    print("=" * 50)

    # Ensure same length
    min_len = min(len(prophet_predictions), len(tft_predictions))
    prophet_pred = prophet_predictions[:min_len]
    tft_pred = tft_predictions[:min_len]
    info_df = prediction_info.iloc[:min_len].copy() if len(prediction_info) > min_len else prediction_info.copy()

    print(f"   📊 Aligning predictions: {min_len:,} samples")

    # Determine holiday flags using date-based detection
    holiday_dates = [
        '2012-11-23',  # Thanksgiving 2012
        '2012-11-30',  # Black Friday 2012
        '2012-12-28',  # Christmas 2012
        '2013-02-08',  # Super Bowl 2013
        '2013-09-06',  # Labor Day 2013
        '2013-11-29',  # Thanksgiving 2013
        '2013-12-06',  # Week before Christmas 2013
        '2013-12-27'   # Christmas 2013
    ]
    holiday_dates = pd.to_datetime(holiday_dates)

    # Check if prediction dates match holiday dates
    pred_dates = pd.to_datetime(info_df['Date'])
    is_holiday = pred_dates.isin(holiday_dates).values

    print(f"   📅 Found {is_holiday.sum()} holiday predictions from {len(holiday_dates)} defined holidays")

    # Apply adaptive weighting
    hybrid_predictions = np.zeros_like(tft_pred, dtype=float)

    # Holiday periods: 60% Prophet + 40% TFT
    holiday_mask = is_holiday
    regular_mask = ~is_holiday

    if np.any(holiday_mask):
        hybrid_predictions[holiday_mask] = (
            0.60 * prophet_pred[holiday_mask] +
            0.40 * tft_pred[holiday_mask]
        )

    # Regular days: 65% TFT + 35% Prophet
    if np.any(regular_mask):
        hybrid_predictions[regular_mask] = (
            0.65 * tft_pred[regular_mask] +
            0.35 * prophet_pred[regular_mask]
        )

    # Ensure no negative predictions
    hybrid_predictions[hybrid_predictions < 0] = 0

    # Report weighting statistics
    n_holiday = np.sum(holiday_mask)
    n_regular = np.sum(regular_mask)
    total_points = len(hybrid_predictions)

    print(f"   📊 Weighting applied:")
    print(f"      Holiday points: {n_holiday:,} ({n_holiday/total_points*100:.1f}%) - 60% Prophet + 40% TFT")
    print(f"      Regular points: {n_regular:,} ({n_regular/total_points*100:.1f}%) - 65% TFT + 35% Prophet")

    print("✅ STEP 7 COMPLETE!")
    return hybrid_predictions, info_df

def step8_generate_submission(hybrid_predictions, prediction_info, filename='submission_hybrid.csv'):
    """
    Step 8: Generate submission file
    Returns: submission file path
    """
    print("\n📝 STEP 8: Generating submission file...")
    print("=" * 50)

    # Create submission DataFrame
    submission_df = pd.DataFrame({
        'Store': prediction_info['Store'].astype(int),
        'Dept': prediction_info['Dept'].astype(int),
        'Date': pd.to_datetime(prediction_info['Date']).dt.strftime('%Y-%m-%d'),
        'Weekly_Sales': hybrid_predictions
    })

    # Create Id column as required: Store_Dept_Date
    submission_df['Id'] = (submission_df['Store'].astype(str) + '_' +
                         submission_df['Dept'].astype(str) + '_' +
                         submission_df['Date'])

    # Select only required columns and reorder
    submission_final = submission_df[['Id', 'Weekly_Sales']].copy()

    # Save submission file
    submission_final.to_csv(filename, index=False)

    print(f"   ✅ Submission file saved: {filename}")
    print(f"   📊 Submission shape: {submission_final.shape}")
    print(f"   📊 Sample predictions:")
    print(submission_final.head(10))

    print(f"\n📊 Final Statistics:")
    print(f"   Total predictions: {len(hybrid_predictions):,}")
    print(f"   Mean prediction: ${np.mean(hybrid_predictions):,.2f}")
    print(f"   Std prediction: ${np.std(hybrid_predictions):,.2f}")
    print(f"   Min prediction: ${np.min(hybrid_predictions):,.2f}")
    print(f"   Max prediction: ${np.max(hybrid_predictions):,.2f}")

    print("✅ STEP 8 COMPLETE!")
    print("🎉 ALL STEPS FINISHED! Submission file is ready!")

    return filename


In [7]:
train_full, test_full = step1_load_and_prepare_data()

📊 STEP 1: Loading and preparing data...
   📂 Loading from local files...
   📈 Train data: (421570, 5)
   📉 Test data: (115064, 4)
   📊 Features data: (8190, 12)
   🏪 Stores data: (45, 3)

🧹 Merging training data and initial cleaning...
🧹 Merging test data...
   🗑️ Removed 1358 rows with negative Weekly_Sales from training.
   ⚠️ Adding missing IsHoliday column to test data with default value False
   Filling missing dates and sales for time series continuity in training data...
   Filled 26507 NaN Weekly_Sales values with 0 for series continuity.
   ✅ Merged and cleaned training data: (446719, 27)
   ✅ Merged and cleaned test data: (115064, 17)
   📅 Train date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
   📅 Test date range: 2012-11-02 00:00:00 to 2013-07-26 00:00:00
✅ STEP 1 COMPLETE!


In [8]:
prophet_predictions, prediction_info = step2_train_prophet_and_predict(train_full, test_full)


📈 STEP 2: Training Prophet and generating predictions...
🚀 Running Standalone Prophet Inference

📊 Step 1: Data preprocessing for inference...
🔄 Getting preprocessed data for inference...
📊 Loading datasets for inference...
   📈 Train data: (421570, 5)
   📉 Test data: (115064, 4)
   📊 Features data: (8190, 12)
   🏪 Stores data: (45, 3)
   ✅ Merged train data: (421570, 16)
   ✅ Merged test data: (115064, 16)
   📅 Train date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
   📅 Test date range: 2012-11-02 00:00:00 to 2013-07-26 00:00:00
🔧 Preparing Prophet specific data (holidays) for inference...
✅ Pipeline fitted on training data with holiday-aware settings for inference
🔄 Transforming training data...
✅ Transform complete. Shape: (421570, 5)
🔄 Transforming test data...
✅ Transform complete. Shape: (115064, 3)
✅ Data preprocessing complete!
   📊 Training shape: (421570, 5)
   📊 Test shape: (115064, 3)

📈 Step 2: Training Prophet models on full training data...
📈 Training Prophet mode

In [9]:
prediction_info

,Store,Dept,Date
0,1,1,2012-11-02
1,1,1,2012-11-09
2,1,1,2012-11-16
3,1,1,2012-11-23
4,1,1,2012-11-30
...,...,...,...
115059,45,98,2013-06-28
115060,45,98,2013-07-05
115061,45,98,2013-07-12
115062,45,98,2013-07-19


In [10]:
prophet_predictions

array([33205.36977642, 27730.18630402, 19150.56605114, ...,
         714.25254874,   657.57536744,   630.51019825])

In [11]:
X_train_full, y_train_full, X_test = step3_prepare_tft_data(train_full, test_full)


⚙️ STEP 3: Preparing data for TFT...
   📈 Full training set: 446,719 records
   📉 Test set: 115,064 records
✅ STEP 3 COMPLETE!


In [12]:
pipeline = step4_build_tft_pipeline()


🔧 STEP 4: Building TFT preprocessing pipeline...
   ✅ Pipeline built successfully
✅ STEP 4 COMPLETE!


In [13]:
pipeline = step5_train_tft(pipeline, X_train_full, y_train_full)


🧠 STEP 5: Training TFT model...


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MAE                      | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 512    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 613 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


✅ STEP 5 COMPLETE!
   🎯 TFT model training finished!


In [14]:
tft_predictions = step6_predict_tft(pipeline, X_test)


🔮 STEP 6: Generating TFT predictions...
DEBUG: Found NaNs in futr_df_complete for columns: ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Type_A', 'Type_B', 'Type_C', 'IsHoliday_False', 'IsHoliday_True', 'Store', 'Dept']. Filling with 0.


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

DEBUG: Found NaNs in final y_pred after reindex. Filling with 0.
✅ STEP 6 COMPLETE!
   📊 Generated 115,064 TFT predictions


In [15]:
tft_predictions

array([21702.885  , 44117.098  ,  9480.585  , ..., 53602.484  ,
        6735.399  ,   712.82556], dtype=float32)

In [16]:
hybrid_predictions, final_info = step7_create_hybrid_predictions(prophet_predictions, tft_predictions, prediction_info)


🤝 STEP 7: Creating hybrid predictions...
   📊 Aligning predictions: 115,064 samples
   📅 Found 11890 holiday predictions from 8 defined holidays
   📊 Weighting applied:
      Holiday points: 11,890 (10.3%) - 60% Prophet + 40% TFT
      Regular points: 103,174 (89.7%) - 65% TFT + 35% Prophet
✅ STEP 7 COMPLETE!


In [17]:
final_info

,Store,Dept,Date
0,1,1,2012-11-02
1,1,1,2012-11-09
2,1,1,2012-11-16
3,1,1,2012-11-23
4,1,1,2012-11-30
...,...,...,...
115059,45,98,2013-06-28
115060,45,98,2013-07-05
115061,45,98,2013-07-12
115062,45,98,2013-07-19


In [18]:
hybrid_predictions

array([25728.75442175, 38381.67848766, 12865.07800071, ...,
       35091.60167331,  4608.16065595,   684.01517827])

In [19]:
submission_file = step8_generate_submission(hybrid_predictions, final_info)


📝 STEP 8: Generating submission file...
   ✅ Submission file saved: submission_hybrid.csv
   📊 Submission shape: (115064, 2)
   📊 Sample predictions:
               Id  Weekly_Sales
0  1_1_2012-11-02  25728.754422
1  1_1_2012-11-09  38381.678488
2  1_1_2012-11-16  12865.078001
3  1_1_2012-11-23  26149.145865
4  1_1_2012-11-30  22440.384052
5  1_1_2012-12-07  14083.020084
6  1_1_2012-12-14  30566.675696
7  1_1_2012-12-21  41151.974070
8  1_1_2012-12-28  29961.579507
9  1_1_2013-01-04  27560.349023

📊 Final Statistics:
   Total predictions: 115,064
   Mean prediction: $16,311.99
   Std prediction: $16,773.18
   Min prediction: $0.00
   Max prediction: $395,422.52
✅ STEP 8 COMPLETE!
🎉 ALL STEPS FINISHED! Submission file is ready!
